# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846914624143                   -0.70    4.8         
  2   -7.852331751974       -2.27       -1.53    1.0   23.4ms
  3   -7.852614840788       -3.55       -2.56    1.2   24.2ms
  4   -7.852645986743       -4.51       -2.88    2.5   31.4ms
  5   -7.852646496349       -6.29       -3.16    1.0   23.2ms
  6   -7.852646679685       -6.74       -4.01    1.0   23.2ms
  7   -7.852646686401       -8.17       -5.11    1.5   27.4ms
  8   -7.852646686724       -9.49       -5.48    2.2   30.1ms
  9   -7.852646686729      -11.32       -5.80    1.0   23.3ms
 10   -7.852646686730      -11.94       -6.44    1.2   24.2ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846792095033                   -0.70           4.5         
  2   -7.852525847712       -2.24       -1.64   0.80    2.2    253ms
  3   -7.852636123862       -3.96       -2.72   0.80    1.0   21.2ms
  4   -7.852646379137       -4.99       -3.25   0.80    2.0   27.0ms
  5   -7.852646667319       -6.54       -4.03   0.80    1.8   25.0ms
  6   -7.852646686371       -7.72       -4.67   0.80    1.8   25.9ms
  7   -7.852646686706       -9.48       -5.49   0.80    1.8   25.3ms
  8   -7.852646686730      -10.63       -6.53   0.80    2.0   26.9ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.388475e+01     3.617397e+00
 * time: 0.4410591125488281
     1     1.096726e+00     1.685084e+00
 * time: 0.6350610256195068
     2    -1.134171e+00     1.949813e+00
 * time: 0.6584711074829102
     3    -3.824264e+00     1.548542e+00
 * time: 0.6913111209869385
     4    -5.116729e+00     1.536114e+00
 * time: 0.7242169380187988
     5    -6.938011e+00     9.218547e-01
 * time: 0.7568809986114502
     6    -7.298856e+00     1.049312e+00
 * time: 0.8755130767822266
     7    -7.653258e+00     3.851928e-01
 * time: 0.8988039493560791
     8    -7.756418e+00     1.014988e-01
 * time: 0.9215600490570068
     9    -7.806572e+00     1.069192e-01
 * time: 0.9441831111907959
    10    -7.823649e+00     1.282912e-01
 * time: 0.9670050144195557
    11    -7.841769e+00     5.565561e-02
 * time: 0.989825963973999
    12    -7.847373e+00     5.991220e-02
 * time: 1.0127809047698975
    13    -7.848547e+00     4.165889e-02
 * time: 1.03600311279

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846758090796                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645867100                   -1.64         
  2   -7.852646686730       -6.09       -3.70    1.61s
  3   -7.852646686730      -13.21       -7.22    161ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.654088307247456e-7
|ρ_newton - ρ_scfv| = 4.7164194216433393e-7
|ρ_newton - ρ_dm|   = 1.3143153367958464e-9
